# Drift Diffusion Model Project

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

## Figure 1

The drift diffusion model is described by:

$$
x_{j,1} = x_0 + \epsilon_{j,0} \\
x_{j,t+1} = x_{j,t} + \beta_{c(j)} + \epsilon_{j,t} \\
\epsilon_{j,t} \sim \mathcal{N}(0,\omega^2) \\
y_{j,t}|t < \tau_j \sim \mbox{Poisson}(\log(1+\exp(\gamma x_t)) \Delta_t) \\
y_{j,t}|t \geq \tau_j \sim \mbox{Poisson}(\log(1+\exp(\gamma)) \Delta_t)
$$

<ul>
<li>$x$ is the variable that drifts. $\beta$ is the coherence ($c$) dependent drift rate and $\epsilon$ is the noise (normal with variance $\omega^2$)</li>
<li>$\tau_j$ is the (absorbing) boundary crossing time. Before $\tau_j$, firing rate depends on $x_t$ and the maximum firing rate $\gamma$.</li>
<li>$y_{j,t}$ are the action potential times.</li>
</ul>

The function $\log(1+\exp(\gamma x_t))$ is the "transfer function" that determines the neuron's firing rate based on the state of the accumulation process ($x_t$). First, let's plot this transfer function to see how it behaves:

In [ ]:
# x_t has an absorbing bound at 1.0
xt = np.arange(-.5, 1.05, .05)
# gamma specifies the maximum firing rate
gamma = 55

firing_rate = []
for i in range(len(xt)):
    x = xt[i]
    firing_rate.append( math.log(1 + math.exp(gamma*x)) )

plt.rcParams['figure.figsize'] = [6,6]
f, a = plt.subplots()
a.plot([0, 1], [0, gamma], '--', color='#888888')
a.plot(xt, firing_rate)
a.set_xlabel('Evidence ($x_t$)')
a.set_ylabel('Firing Rate (Hz)')
a.set_title('Transfer Function')
plt.show()

### Drift Diffusion Model

Model parameters similar to a true LIP neurons are given in the figure legend for supplemental figure S5. We will use these parameters as a starting point and modify to match Figure 1 as needed.

In [ ]:
beta = [-4.7e-3, -2.4e-4, -1.3e-3, 6e-4, 3.4e-3]
x0 = .72
w2 = 1.7e-3
gamma = 55

Reproduce Figure 1B by modeling the drift in spike rate and stochastically generating spikes.

For modeling, Latimer et al. simulated the diffusion process at a time resolution of $\Delta_t = 10 \mbox{ ms}$ and spike generation at a time interval of $0.2 \mbox{ ms}$. To help Python run faster, it is fine to use a time interval of $1 \mbox{ ms}$ to model spike times.

In [ ]:
Dt = 10
dt = 1 #.2

### Problem 1

<b>Implement the drift diffusion algorithm to reproduce Figure 1B</b>. I find that Figure 1B can be approximated by assuming that the drift rates ($\beta$) are 3 times greater than the values given in Fig S5. 

In [ ]:
# prep the plot
plt.rcParams['figure.figsize'] = [8,10]
a1 = plt.subplot(2,1,1)
a2 = plt.subplot(2,1,2)
colors = ['#f3613e', '#b05639', '#656769', '#456b94', '#69a2d8']

# number of trials to simulate. Figure 1B has 5 trials per condition
num_trials = 5

# create list of time points
t_min = 0
t_max = 600
time = np.arange(t_min, t_max+dt, dt)

# simulate for low, zero, and high coherence
for c in [0,2,4]:
    for trial in range(num_trials):
        # initialize lists for this trial
        x = []
        spike_rate = []
        spikes = []

        for t in time:
            # only update the diffusion process every Dt msec
            if round(t,1) % Dt == 0:
                # don't start the diffusion process before 200ms post stimulus
                if t <= 200:
                    x.append(x0)

                # implement the diffusion process

            # determine whether a spike occurred at t and store time if so
            

        a1.plot(np.arange(t_min,t_max+dt,Dt), spike_rate, color=colors[c])
        a2.plot(spikes, np.ones(len(spikes))*((c/2*num_trials)+trial+1), '.', color=colors[c])

a1.set_ylim( (0, 60) )
a1.plot([200,200], a1.get_ylim(), ':', color='#cccccc')
a2.plot([200,200], a2.get_ylim(), ':', color='#cccccc')

a1.set_ylabel('spike rate (sp/s)')
a2.set_ylabel('trial number')
a2.set_xlabel('time (ms)')

plt.show()

### Stepping Model

The stepping model is described by the following equations:

$$
z_j \sim \mbox{Negative Binomial}(p_{c(j)}, r) \\
P(d_j=1) = \phi_{c(j)} \\
P(d_j=2) = 1 - \phi_{c(j)} \\
y_{j,t}|t \leq z_j \sim \mbox{Poisson}(\alpha_0 \Delta_t) \\
y_{j,t}|t > z_j \sim \mbox{Poisson}(\alpha_{d_j} \Delta_t)
$$

<ul>
<li>$z_j$ is the stepping time. The probability function that detemines step times differs by condition $c(j)$.</li>
<li>$d_j$ is the stepping destination. $d_j=1$ corresponds to a down step and $d_j=2$ corresponds to an up step.</li>
<li>$y_{j,t}$ is the probability of an action potential at any given time $t$ where $\alpha$ is the firing rate in that state and state 0 is the initial state.</li>
</ul>

Python does not come with a negative binomial function that works with non-integer values. However, the internet has one:

In [ ]:
# From http://www.nehalemlabs.net/prototype/blog/2013/11/11/negative-binomial-with-continuous-parameters-in-python/

import scipy.special as special
import scipy.optimize as optimize
import numpy as np
import mpmath
 
class negBin(object):
    def __init__(self, p = 0.1, r = 10):
        nbin_mpmath = lambda k, p, r: mpmath.gamma(k + r)/(mpmath.gamma(k+1)*mpmath.gamma(r))*np.power(1-p, r)*np.power(p, k)
        self.nbin = np.frompyfunc(nbin_mpmath, 3, 1)
        self.p = p
        self.r = r
 
    def mleFun(self, par, data, sm):
        '''
        Objective function for MLE estimate according to
        https://en.wikipedia.org/wiki/Negative_binomial_distribution#Maximum_likelihood_estimation

        Keywords:
        data -- the points to be fit
        sm -- \sum data / len(data)
        '''
        p = par[0]
        r = par[1]
        n = len(data)
        f0 = sm/(r+sm)-p
        f1 = np.sum(special.psi(data+r)) - n*special.psi(r) + n*np.log(r/(r+sm))
        return np.array([f0, f1])

    def fit(self, data, p = None, r = None):
        if p is None or r is None:
            av = np.average(data)
            va = np.var(data)
            r = (av*av)/(va-av)
            p = (va-av)/(va)
        sm = np.sum(data)/len(data)
        x = optimize.fsolve(self.mleFun, np.array([p, r]), args=(data, sm))
        self.p = x[0]
        self.r = x[1]

    def pdf(self, k):
        try:
            return self.nbin(k, self.p, self.r).astype('float64')
        except:
            return self.nbin(k, self.p, self.r)

Model parameter for an example LIP neuron are given in the legend for Figure S5. The firing rate parameters ($\alpha$) need to be changed to match Figure 1C. 

In [ ]:
alpha = [25, 10, 55] #[4.1, .57, 41.0]
phi = [.10, .30, .71, .82, .98]
p = [0.99, 0.98, 0.98, 0.975, 0.97]
r = 1.05

We can now take a look at the negative binomial distribution to get an idea about how the stepping process is modeled. Note that the distribution of step times is plotted in Figure 2B.

I find that Figure 1C is better approximated by letting $p_{c(j)}=0.995$ for all $c(j)$ and letting $r=1.5$.

In [ ]:
colors = ['#f3613e', '#b05639', '#656769', '#456b94', '#69a2d8']

plt.rcParams['figure.figsize'] = [8,6]
f,a = plt.subplots()

for i in range(len(p)):
    neg_bin = negBin(p=p[i], r=r)
    #neg_bin = negBin(p=.995, r=1.5)

    xstep = 10
    xbin = np.arange(0,1200+xstep,xstep)

    a.plot( xbin, neg_bin.pdf(xbin), color=colors[i] )

a.set_xlabel('time (msec)')
a.set_ylabel('P(step)')
a.set_title('Negative Binomial Distribution')
plt.show()

### Problem 2

<b>Implement the stepping model to reproduce Figure 1C:</b>

In [ ]:
# prep the plot
plt.rcParams['figure.figsize'] = [8,10]
a1 = plt.subplot(2,1,1)
a2 = plt.subplot(2,1,2)
colors = ['#f3613e', '#b05639', '#656769', '#456b94', '#69a2d8']


# number of trials to simulate. Figure 1B has 5 trials per condition
num_trials = 5

# create list of time points
t_min = 0
t_max = 600
time = np.arange(t_min, t_max+dt, dt)

# simulate for low, zero, and high coherence
for c in [0,2,4]:
    #neg_bin = negBin(p=p[c], r=r)
    neg_bin = negBin(p=.995, r=1.5)
    
    for trial in range(num_trials):
        # initialize trial variables
        state = 0
        spike_rate = []
        spike_rate_times = []
        spikes = []

        for t in time:
            # test whether a step should occur
            
            # if so, select next state
            
            # store modeled spike_rate for plotting

            # test if an action potential should occur
            
            # if so, store the time in spikes
        
        
        a1.plot(spike_rate_times, spike_rate, color=colors[c])
        a2.plot(spikes, np.ones(len(spikes))*((c/2*num_trials)+trial+1), '.', color=colors[c])


a1.set_ylim( (0, 60) )
a1.plot([200,200], a1.get_ylim(), ':', color='#cccccc')
a2.plot([200,200], a2.get_ylim(), ':', color='#cccccc')

a1.set_ylabel('spike rate (sp/s)')
a2.set_ylabel('trial number')
a2.set_xlabel('time (ms)')

plt.show()

## Figure 3A

For this figure we will stimulate spike times and then uses these spikes to generate firing rate plots for the two models, aligned both to stimulus onset and step time.

The following fuction will make your life easier. It converts simulated spike times into an estimated spike rate across time.

In [ ]:
def spike_times_to_rate(spikes, time_step=Dt, t_min=0, t_max=400):
    firing_rate = np.zeros( int( (t_max-t_min)/Dt ) )
    
    for spike_times in spikes.values():
        for s in spike_times:
            i = int( (s-t_min)/Dt )
            if i>=0 and i<len(firing_rate): firing_rate[i] += 1
    
    firing_rate /= len(spikes)  # normalize to probability of spikes per bin
    firing_rate /= Dt           # convert to per msec
    firing_rate *= 1000         # convert to per sec (Hz)
    
    return firing_rate

### Drift Diffusion Model

### Problem 3

Model the stimulus-aligned firing rates for the diffusion model:

In [ ]:
# prep the plot
plt.rcParams['figure.figsize'] = [8,10]
a1 = plt.subplot(2,1,1)
a2 = plt.subplot(2,1,2)
colors = ['#f3613e', '#b05639', '#656769', '#456b94', '#69a2d8']

# simulate more trials to average over trial-level noise 
num_trials = 100

# this time we need to keep track of spikes for all trials. we do
# this by storing lists in a dict, where the keys are the trial numbers.
# this allows us to average over trials at the end to get mean firing rate.
spikes = {}

# create list of time points
t_min = -100
t_max = 600
time = np.arange(t_min, t_max+dt, dt)

# simulate for all conditions
for c in range(5):
    for trial in range(num_trials):
        # initialize trial variables
        x = x0
        spikes[trial] = []

        for t in time:
            # only update the diffusion process every Dt msec
            if round(t,1) % Dt == 0:
                # update x

            # determine whether an action potential should occur
            
            # if so, append time to spikes[trial]
        
        # only plot spikes for the first 5 trials
        if trial < 5:
            a1.plot(spikes[trial], np.ones(len(spikes[trial]))*((c*5)+trial+1), '.', color=colors[c])
        

    rate = spike_times_to_rate(spikes, t_min=t_min, t_max=t_max)
    a2.plot(np.arange(t_min,t_max,Dt), rate, color=colors[c])

    
a1.plot([0,0], a1.get_ylim(), ':k', color='#cccccc')
a2.plot([0,0], a2.get_ylim(), ':k', color='#cccccc')

a1.set_ylabel('trial number')
a2.set_ylabel('spike rate (sp/s)')
a2.set_xlabel('time (ms)')

plt.show()

### Stepping Model

### Problem 4

Plot modeled stimulus-aligned firing rates for the stepping model.

In [ ]:
# prep the plot
plt.rcParams['figure.figsize'] = [8,10]
a1 = plt.subplot(2,1,1)
a2 = plt.subplot(2,1,2)
colors = ['#f3613e', '#b05639', '#656769', '#456b94', '#69a2d8']

# simulate more trials to average over trial-level noise 
num_trials = 50

# this time we need to keep track of spikes for all trials. we do
# this by storing lists in a dict, where the keys are the trial numbers.
# this allows us to average over trials at the end to get mean firing rate.
spikes = {}

# create list of time points
t_min = -100
t_max = 600
time = np.arange(t_min, t_max+dt, dt)

# simulate for all conditions
for c in range(5):
    #neg_bin = negBin(p=p[c], r=r)
    neg_bin = negBin(p=.995, r=1.5)
    
    for trial in range(num_trials):
        # initialize trial variables
        state = 0
        spikes[trial] = []

        for t in time:
            # determine if a step should occur
            
            # if so, update state variable

            # determine whether an action potential should occur
            
            # if so, store the time in spikes[trial]
                
                
        # only plot spikes for the first 5 trials
        if trial < 5:
            a1.plot(spikes[trial], np.ones(len(spikes[trial]))*((c*5)+trial+1), '.', color=colors[c])
        

    rate = spike_times_to_rate(spikes, t_min=t_min, t_max=t_max)
    a2.plot(np.arange(t_min,t_max,Dt), rate, color=colors[c])

    
a1.plot([0,0], a1.get_ylim(), ':k', color='#cccccc')
a2.plot([0,0], a2.get_ylim(), ':k', color='#cccccc')

a1.set_ylabel('trial number')
a2.set_ylabel('spike rate (sp/s)')
a2.set_xlabel('time (ms)')

plt.show()

### Step Aligned

### Problem 5

Re-run the stepping model, but adjust spike times so that $t=0$ corresponds to the step time.

In [ ]:
# prep the plot
plt.rcParams['figure.figsize'] = [8,10]
a1 = plt.subplot(2,1,1)
a2 = plt.subplot(2,1,2)
colors = ['#f3613e', '#b05639', '#656769', '#456b94', '#69a2d8']

# simulate more trials to average over trial-level noise 
num_trials = 50

# this time we need to keep track of trial variables for all trials
# this allows us to average over trials at the end to get mean firing rate
x = {}
spike_rate = {}
spike_rate_times = {}
spikes = {}

# create list of time points
t_min = -100
t_max = 1000
time = np.arange(t_min, t_max+dt, dt)

# simulate for all conditions
for c in range(5):
    #neg_bin = negBin(p=p[c], r=r)
    neg_bin = negBin(p=.995, r=1.5)
    
    trial = 0
    while trial < num_trials:
        # initialize trial variables
        state = 0
        step_time = 0
        spikes[trial] = []

        for t in time:
            # determine if a step should occur
            
            # if so, update state variable and store step_time

            # determine whether an action potential should occur
            
            # if so, store the time in spikes[trial]
           
        if state == 0:
            # only use this trial if a step occurred. otherwise, nothing to align to
            continue
        else:
            # adjust spike times based on step_time
            
            # only plot spikes for the first 5 trials
            if trial < 5:
                a1.plot(spikes[trial], np.ones(len(spikes[trial]))*((c*5)+trial+1), '.', color=colors[c])
            
            trial += 1

    rate = spike_times_to_rate(spikes, t_min=-100, t_max=300)
    a2.plot(np.arange(-100,300,Dt), rate, color=colors[c])

    
a1.plot([0,0], a1.get_ylim(), ':k', color='#cccccc')
a2.plot([0,0], a2.get_ylim(), ':k', color='#cccccc')

a1.set_ylabel('trial number')
a2.set_ylabel('spike rate (sp/s)')
a2.set_xlabel('time (ms)')

plt.show()